<a href="https://colab.research.google.com/github/Ycy97/PythonTutorial/blob/main/Recommender_system_collaborative_filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
! curl http://files.grouplens.org/datasets/movielens/ml-latest-small.zip -o ml-latest-small.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  1  955k    1 17003    0     0  21785      0  0:00:44 --:--:--  0:00:44 21798
 16  955k   16  160k    0     0    99k      0  0:00:09  0:00:01  0:00:08   99k
 42  955k   42  406k    0     0   152k      0  0:00:06  0:00:02  0:00:04  152k
 67  955k   67  645k    0     0   180k      0  0:00:05  0:00:03  0:00:02  180k
 93  955k   93  895k    0     0   194k      0  0:00:04  0:00:04 --:--:--  194k
100  955k  100  955k    0     0   196k      0  0:00:04  0:00:04 --:--:--  230k


Recommendation System : Collaborative System + Matrix Factorization & Kmeans

In [10]:
import zipfile
with zipfile.ZipFile('ml-latest-small.zip','r') as zip_ref:
  zip_ref.extractall('data')

In [11]:
#import the dataset
import pandas as pd

movies_df = pd.read_csv('data/ml-latest-small/movies.csv')
ratings_df = pd.read_csv('data/ml-latest-small/ratings.csv')

In [12]:
print('The dimensions of movie dataframe are :', movies_df.shape, '\nThe dimensions of ratings dataframe are:', ratings_df.shape)

The dimensions of movie dataframe are : (9742, 3) 
The dimensions of ratings dataframe are: (100836, 4)


In [13]:
#Take a look at movies_df
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [14]:
#Take a look at ratings_df
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [15]:
#Movie ID to movie name mapping
movie_names = movies_df.set_index('movieId')['title'].to_dict()
#print(movie_names)
n_users = len(ratings_df.userId.unique())
n_items = len(ratings_df.movieId.unique())
print("Number of unique users :", n_users)
print("Number of unique movies: ", n_items)
print("The full rating matrix will have:", n_users*n_items,"elements.")
print("--------------------------------------------------------------")
print("Number of ratings:",len(ratings_df))
print("Therefore:", len(ratings_df)/(n_users*n_items) * 100, '% of the matrix is filled.')
print("We have an incredibly sparse matrix to work with here.")
print("And...as you can imagine, as the number of users and product grow, the number of elements will increase by n*2")
print("You are going to need a lot of memory to work with global scale ... storing a full matrix in memory would be a challenge.")
print("One advantage here is that matrix factorization can realize the rating matrix implicitly, thus we dont need all the data.")

Number of unique users : 610
Number of unique movies:  9724
The full rating matrix will have: 5931640 elements.
--------------------------------------------------------------
Number of ratings: 100836
Therefore: 1.6999683055613624 % of the matrix is filled.
We have an incredibly sparse matrix to work with here.
And...as you can imagine, as the number of users and product grow, the number of elements will increase by n*2
You are going to need a lot of memory to work with global scale ... storing a full matrix in memory would be a challenge.
One advantage here is that matrix factorization can realize the rating matrix implicitly, thus we dont need all the data.


In [16]:
import torch
import numpy as np
from torch.autograd import Variable
from tqdm import tqdm_notebook as tqdm

class MatrixFactorization(torch.nn.Module):
  def __init__(self,n_users,n_items,n_factors=20):
    super().__init__()
    #create user embeddings
    self.user_factors = torch.nn.Embedding(n_users,n_factors)# think of this as a lookup table for the input
    #create item embeddings
    self.item_factors = torch.nn.Embedding(n_items,n_factors)# think of this as a lookup table for the input
    self.user_factors.weight.data.uniform_(0, 0.05)
    self.item_factors.weight.data.uniform_(0, 0.05)
  
  def forward(self,data):
    #matrix multiplication
    users, items = data[:,0], data[:,1] #all data from column 0 and column 1
    return (self.user_factors(users)*self.item_factors(items)).sum(1)
  
  def predict(self, user, item):
    return self.forward(user,item)


ModuleNotFoundError: No module named 'torch'

In [ ]:
#Creating the dataloader (necessary for PyTorch)
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader # package that helps transform your data to machine learning readiness

#Note : This isnt "good" practice, in a MLops sense but we'll roll with this since the data is already loaded in memory, should be ratings_df in real application for parameter
class Loader(Dataset):
  def __init__(self):
    self.ratings = ratings_df.copy()

    #Extract all user IDs and movie IDs
    users = ratings_df.userId.unique()
    movies = ratings_df.movieId.unique()

    #--- Producing new continuous IDs for users and movies ---

    #Unique values : index
    self.userid2idx = {o:i for i,o in enumerate(users)}
    self.movieid2idx = {o:i for i,o in enumerate(movies)}

    #Obtained continuous ID for users and movies
    self.idx2userid = {i:o for o,i in self.userid2idx.items()}
    self.idx2movieid = {i:o for o,i in self.movieid2idx.items()}

    #return the id from the indexed values as noted in the lambda function down below
    self.ratings.movieId = ratings_df.movieId.apply(lambda x: self.movieid2idx[x])
    self.ratings.userId = ratings_df.userId.apply(lambda x: self.userid2idx[x])

    self.x = self.ratings.drop(['rating','timestamp'],axis=1).values
    self.y = self.ratings['rating'].values
    self.x, self.y = torch.tensor(self.x), torch.tensor(self.y) #Transforms the data to tensors (ready for torch models.)
  
  def __getitem__(self,index):
    return (self.x[index], self.y[index])

  def __len__(self):
    return len(self.ratings)



In [ ]:
num_epochs = 128
cuda = torch.cuda.is_available()

print("Is running on GPU:", cuda)

model = MatrixFactorization(n_users, n_items, n_factors=8)
print(model)
for name, param in model.named_parameters():
  if param.requires_grad:
    print(name, param.data)

#GPU enable if you have a GPU...
if cuda:
  model  = model.cuda()

#MSE los
loss_fn = torch.nn.MSELoss()

#ADAM optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

#Train data
train_set = Loader()
train_loader = DataLoader(train_set, 128, shuffle=True)


In [ ]:
for it in tqdm(range(num_epochs)):
  losses = []
  for x,y in train_loader:
    if cuda:
      x,y = x.cuda(), y.cuda()
      optimizer.zero_grad()
      outputs = model(x)
      loss = loss_fn(outputs.squeeze(), y.type(torch.float32))
      losses.append(loss.item())
      loss.backward()
      optimizer.step()
  print("iter #{}".format(it), "Loss:", sum(losses)/len(losses))

In [ ]:
#By training the model, we will have tuned latent factors for movies and users.
c = 0
uw = 0
iw = 0
for name, param in model.named_parameters():
  if param.requires_grad:
    print(name, param.data)
    if c==0:
      uw = param.data
      c+=1
    else:
      i = param.data
      

In [ ]:
trained_movie_embeddings = model.item_factors.weight.data.cpu().numpy()

In [ ]:
trained_movie_embeddings # unique movie factor weights

In [ ]:
from sklearn.cluster import KMeans
#Fit the clusters based on the movie weights
kmeans = KMeans(n_clusters=10, random_state=0).fit(trained_movie_embeddings)

In [ ]:
#it can be seen here that the movies that are in the same cluster tend to have similar genres. also note that the algorithm is unfamiliar with the movie
#name and only obtained the relationships by looking at the numbers representing how users have responded to the movie selections

for cluster in range(10):
  print("Cluster #{}".format(cluster))
  movs = []
  for movidx in np.where(kmeans.labels_ == cluster)[0]:
    movid = train_set.idx2movieid[movidx]
    rat_count = ratings_df.loc[ratings_df['movieId']==movid].count()[0]
    movs.append((movie_names[movid],rat_count))
  for mov in sorted(movs, key=lambda tup: tup[1], reverse=True)[:10]:
    print("\t", mov[0])